# Data Exploration

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("vgsales.csv",  header=0)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16598 non-null  int64  
 1   Name          16598 non-null  object 
 2   Platform      16598 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16598 non-null  object 
 5   Publisher     16540 non-null  object 
 6   NA_Sales      16598 non-null  float64
 7   EU_Sales      16598 non-null  float64
 8   JP_Sales      16598 non-null  float64
 9   Other_Sales   16598 non-null  float64
 10  Global_Sales  16598 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.4+ MB


In [4]:
df.shape

(16598, 11)

In [5]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,259,Asteroids,2600,1980.0,Shooter,Atari,4.00,0.26,0.0,0.05,4.31
1,545,Missile Command,2600,1980.0,Shooter,Atari,2.56,0.17,0.0,0.03,2.76
2,1768,Kaboom!,2600,1980.0,Misc,Activision,1.07,0.07,0.0,0.01,1.15
3,1971,Defender,2600,1980.0,Misc,Atari,0.99,0.05,0.0,0.01,1.05
4,2671,Boxing,2600,1980.0,Fighting,Activision,0.72,0.04,0.0,0.01,0.77


In [6]:
numeric_cols = df.select_dtypes(include = ['int64', 'float64']).columns
print(len(numeric_cols),"\n",numeric_cols)

7 
 Index(['Rank', 'Year', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales',
       'Global_Sales'],
      dtype='object')


In [7]:
object_cols = df.select_dtypes(include = 'object').columns
print(len(object_cols),"\n",object_cols)

4 
 Index(['Name', 'Platform', 'Genre', 'Publisher'], dtype='object')


# Duplicate detection and removal

In [8]:
num_duplicate_rows = df.duplicated().sum()
print("Number of duplicate rows:", num_duplicate_rows)

Number of duplicate rows: 0


# Handling missing values

## Object columns

In [9]:
print(f"Total rows in df:", len(df))
for col in object_cols:
    print(f"Column name:", col)
    num_rows = df[col].notnull().sum()
    print(f"Total non null values in column:", num_rows)
    num_missing = df[col].isna().sum()
    print(f"Missing values in column:", num_missing)
    missing_pct = (num_missing/num_rows) * 100
    if missing_pct > 0:
        print(f"Percentage of column with missing values:", missing_pct)
    print()

Total rows in df: 16598
Column name: Name
Total non null values in column: 16598
Missing values in column: 0

Column name: Platform
Total non null values in column: 16598
Missing values in column: 0

Column name: Genre
Total non null values in column: 16598
Missing values in column: 0

Column name: Publisher
Total non null values in column: 16540
Missing values in column: 58
Percentage of column with missing values: 0.35066505441354295



In [10]:
df.fillna({"Publisher": "Unknown"}, inplace = True)
df["Publisher"].isna().sum()

np.int64(0)

## Numeric columns

In [11]:
print(f"Total rows in df:", len(df))
for col in numeric_cols:
    print(f"Column name:", col)
    num_rows = df[col].notnull().sum()
    print(f"Total non null values in column:", num_rows)
    num_missing = df[col].isna().sum()
    print(f"Missing values in column:", num_missing)
    missing_pct = (num_missing/num_rows) * 100
    if missing_pct > 0:
        print(f"Percentage of column with missing values:", missing_pct)
    print()

Total rows in df: 16598
Column name: Rank
Total non null values in column: 16598
Missing values in column: 0

Column name: Year
Total non null values in column: 16327
Missing values in column: 271
Percentage of column with missing values: 1.6598272799657012

Column name: NA_Sales
Total non null values in column: 16598
Missing values in column: 0

Column name: EU_Sales
Total non null values in column: 16598
Missing values in column: 0

Column name: JP_Sales
Total non null values in column: 16598
Missing values in column: 0

Column name: Other_Sales
Total non null values in column: 16598
Missing values in column: 0

Column name: Global_Sales
Total non null values in column: 16598
Missing values in column: 0



In [12]:
df = df.dropna(subset=["Year"])
df.reset_index(drop=True, inplace=True)

In [13]:
df["Year"] = df["Year"].astype(int)

In [14]:
df.shape

(16327, 11)

# Confirming accuracy of Global_Sales

In [15]:
df["Calculated_Sales"] = df["NA_Sales"] + df["EU_Sales"] + df["JP_Sales"] + df["Other_Sales"]

In [16]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Calculated_Sales
0,259,Asteroids,2600,1980,Shooter,Atari,4.00,0.26,0.0,0.05,4.31,4.31
1,545,Missile Command,2600,1980,Shooter,Atari,2.56,0.17,0.0,0.03,2.76,2.76
2,1768,Kaboom!,2600,1980,Misc,Activision,1.07,0.07,0.0,0.01,1.15,1.15
3,1971,Defender,2600,1980,Misc,Atari,0.99,0.05,0.0,0.01,1.05,1.05
4,2671,Boxing,2600,1980,Fighting,Activision,0.72,0.04,0.0,0.01,0.77,0.77


## Identifying inaccurate Global_Sales values

In [17]:
mask = ~np.isclose(df["Global_Sales"], df["Calculated_Sales"], atol=0.01)
df_updates = df.loc[mask, ["Name", "Global_Sales", "Calculated_Sales"]]
print(df_updates)

                                 Name  Global_Sales  Calculated_Sales
1259                         Tekken 3          7.16              7.18
7453        Yu-Gi-Oh! GX: Tag Force 2          0.42              0.44
7674                   Call of Juarez          0.19              0.17
8426       Need For Speed: Undercover          1.40              1.42
9765              Assassin's Creed II          5.27              5.25
9972                              Wet          0.63              0.61
12518         Need for Speed: The Run          1.38              1.36
13639            Kid Icarus: Uprising          1.28              1.26
14325                  Rocksmith 2014          1.00              0.98
14992  One Piece: Unlimited World Red          0.30              0.32


## Overriding the wrong values with the accurately calculated ones

In [18]:
updated_idx = df.index[mask]
df.loc[updated_idx, "Global_Sales"] = df.loc[updated_idx, "Calculated_Sales"]

In [19]:
df_updates = df.loc[updated_idx, ["Name", "Global_Sales", "Calculated_Sales"]]
print(df_updates)

                                 Name  Global_Sales  Calculated_Sales
1259                         Tekken 3          7.18              7.18
7453        Yu-Gi-Oh! GX: Tag Force 2          0.44              0.44
7674                   Call of Juarez          0.17              0.17
8426       Need For Speed: Undercover          1.42              1.42
9765              Assassin's Creed II          5.25              5.25
9972                              Wet          0.61              0.61
12518         Need for Speed: The Run          1.36              1.36
13639            Kid Icarus: Uprising          1.26              1.26
14325                  Rocksmith 2014          0.98              0.98
14992  One Piece: Unlimited World Red          0.32              0.32


In [20]:
df.drop(columns=["Calculated_Sales"], inplace=True)

In [21]:
#Confirming the changes were made
print(df.loc[updated_idx, ["Name", "Global_Sales", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]])

                                 Name  Global_Sales  NA_Sales  EU_Sales  \
1259                         Tekken 3          7.18      3.27      2.22   
7453        Yu-Gi-Oh! GX: Tag Force 2          0.44      0.03      0.20   
7674                   Call of Juarez          0.17      0.14      0.02   
8426       Need For Speed: Undercover          1.42      0.38      0.08   
9765              Assassin's Creed II          5.25      3.10      1.56   
9972                              Wet          0.61      0.22      0.27   
12518         Need for Speed: The Run          1.36      0.63      0.58   
13639            Kid Icarus: Uprising          1.26      0.48      0.35   
14325                  Rocksmith 2014          0.98      0.51      0.27   
14992  One Piece: Unlimited World Red          0.32      0.09      0.12   

       JP_Sales  Other_Sales  
1259       1.40         0.29  
7453       0.10         0.11  
7674       0.00         0.01  
8426       0.03         0.93  
9765       0.08    

# Grouping Platforms with little frequency

In [22]:
platform_counts = df["Platform"].value_counts()
print(platform_counts)

Platform
DS      2133
PS2     2127
PS3     1304
Wii     1290
X360    1235
PSP     1197
PS      1189
PC       943
GBA      811
XB       803
GC       542
3DS      500
PSV      412
PS4      336
N64      316
SNES     239
XOne     213
SAT      173
WiiU     143
2600     116
NES       98
GB        97
DC        52
GEN       27
NG        12
WS         6
SCD        6
3DO        3
TG16       2
GG         1
PCFX       1
Name: count, dtype: int64


In [23]:
rare_platforms = platform_counts[platform_counts < 50].index
df["Platform"] = df["Platform"].replace(rare_platforms, "Other")

In [24]:
df["Platform"].value_counts()

Platform
DS       2133
PS2      2127
PS3      1304
Wii      1290
X360     1235
PSP      1197
PS       1189
PC        943
GBA       811
XB        803
GC        542
3DS       500
PSV       412
PS4       336
N64       316
SNES      239
XOne      213
SAT       173
WiiU      143
2600      116
NES        98
GB         97
Other      58
DC         52
Name: count, dtype: int64

# Identifying errors in Columns

## Identifying errors in Genre column

In [25]:
df["Genre"].value_counts()

Genre
Action          3253
Sports          2304
Misc            1710
Role-Playing    1471
Shooter         1282
Adventure       1276
Racing          1226
Platform         876
Simulation       851
Fighting         836
Strategy         671
Puzzle           571
Name: count, dtype: int64

## Identifying errors in Year column

In [26]:
invalid_years = df[(df["Year"] < 1970) | (df["Year"] > 2025)]
print(len(invalid_years))
print(invalid_years)

0
Empty DataFrame
Columns: [Rank, Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales]
Index: []


## Identifying errors in Sales columns

In [27]:
invalid_sales = df[(df[["NA_Sales","EU_Sales","JP_Sales","Other_Sales","Global_Sales"]] < 0).any(axis=1)]
print(len(invalid_sales))
print(invalid_sales)

0
Empty DataFrame
Columns: [Rank, Name, Platform, Year, Genre, Publisher, NA_Sales, EU_Sales, JP_Sales, Other_Sales, Global_Sales]
Index: []


## Identifying errors in Rank column

In [28]:
len(df)

16327

In [29]:
df["Rank"].max()

np.int64(16600)

In [30]:
df["Rank"].min()

np.int64(1)

In [31]:
df["Rank"].isna().sum()

np.int64(0)

In [32]:
df["Rank"] = df["Global_Sales"].rank(ascending=False, method="first").astype(int)

In [33]:
df["Rank"].max()

np.int64(16327)

In [34]:
df["Rank"].min()

np.int64(1)

## Identifying errors in Name column

In [35]:
df["Name"].nunique()

11360

## Identifying errors in Publisher column

In [36]:
df["Publisher"].isna().sum()

np.int64(0)

In [37]:
df["Publisher"].value_counts().sum()

np.int64(16327)

# Feature Engineering: Identifying Exclusive vs Multi-platform Games

In [38]:
name_platform_counts = df.groupby("Name")["Platform"].nunique()
df["Num_Platforms"] = df["Name"].map(name_platform_counts)

In [39]:
df["Exclusive"] = (df["Num_Platforms"] == 1).astype(int)

# Saving cleaned dataframe

In [40]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Num_Platforms,Exclusive
0,256,Asteroids,2600,1980,Shooter,Atari,4.00,0.26,0.0,0.05,4.31,2,0
1,541,Missile Command,2600,1980,Shooter,Atari,2.56,0.17,0.0,0.03,2.76,2,0
2,1743,Kaboom!,2600,1980,Misc,Activision,1.07,0.07,0.0,0.01,1.15,1,1
3,1945,Defender,2600,1980,Misc,Atari,0.99,0.05,0.0,0.01,1.05,4,0
4,2622,Boxing,2600,1980,Fighting,Activision,0.72,0.04,0.0,0.01,0.77,2,0


In [41]:
df.shape

(16327, 13)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16327 entries, 0 to 16326
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           16327 non-null  int64  
 1   Name           16327 non-null  object 
 2   Platform       16327 non-null  object 
 3   Year           16327 non-null  int64  
 4   Genre          16327 non-null  object 
 5   Publisher      16327 non-null  object 
 6   NA_Sales       16327 non-null  float64
 7   EU_Sales       16327 non-null  float64
 8   JP_Sales       16327 non-null  float64
 9   Other_Sales    16327 non-null  float64
 10  Global_Sales   16327 non-null  float64
 11  Num_Platforms  16327 non-null  int64  
 12  Exclusive      16327 non-null  int64  
dtypes: float64(5), int64(4), object(4)
memory usage: 1.6+ MB


In [43]:
df.to_csv("vgsales_clean.csv", index=False)